In [21]:
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt

In [22]:
# Starting index for the pictures
pic_index = 0


In [26]:
IMG_SIZE = 224
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2

train_set = tf.keras.utils.image_dataset_from_directory(
    subset="training",
    directory='/Users/e123e/Desktop/Data-OD/D-set',
    labels='inferred',
    label_mode='categorical',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    validation_split=VALIDATION_SPLIT,
)

valid_set = tf.keras.utils.image_dataset_from_directory(
    subset="validation",
    directory='/Users/e123e/Desktop/Data-OD/D-set',
    labels='inferred',
    label_mode='categorical',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    validation_split=VALIDATION_SPLIT,
)

Found 24610 files belonging to 3 classes.
Using 19688 files for training.
Found 24610 files belonging to 3 classes.
Using 4922 files for validation.


In [27]:
train_set.class_names

['test', 'train', 'valid']

In [8]:
#!pip install ultralytics

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/c1/45/68650eb444bbe0bc762aa2a5007f9f4ff613709165b579e1760cd0c65fbd/ultralytics-8.1.8-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/40.2 kB ? eta -:--:--
     ---------- ----------------------------- 10.2/40.2 kB ? eta -:--:--
     ------------------- ------------------ 20.5/40.2 kB 165.2 kB/s eta 0:00:01
     ----------------------------- -------- 30.7/40.2 kB 217.9 kB/s eta 0:00:01
     -------------------------------------- 40.2/40.2 kB 212.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/709.4 kB ? eta -:--:--
   - -------------------------------------- 30.7/709.4 kB 1.3 MB/s eta 0:00:01
   -- ------------------------------------ 41.0/709.4 kB 960.0 kB/s eta 0:00:01
   -------- ------------------------------- 143.4/709.4 kB 1.2 MB/s eta 0:00:01
   ---------- ----------------------------- 194.6/709.4 kB 1.2 MB/s eta 0:00:01
   ---------------

In [9]:
# for object detection
from ultralytics import YOLO

In [10]:
# create the neural network model
model = YOLO("yolov8m.pt")

100%|██████████| 49.7M/49.7M [00:39<00:00, 1.32MB/s]


In [20]:
#results = model.predict("1.jpg")
results = model.train(data='coco128.yaml', epochs=1, imgsz=640, device=[0, 1])
#-1-_png.rf.5d9ff973229c8c7fbb33dbefa88141e3.jpg

Ultralytics YOLOv8.1.8 🚀 Python-3.11.5 torch-2.1.2+cpu 


ValueError: Invalid CUDA 'device=0,1' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: 0,1
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.
